In [18]:
using Dates
now()

2019-03-12T21:40:36.932

Speaker: [Marcelo Forets](http://main.marcelo-forets.fr/), Universidad de la República.
Centro Universitario Regional del Este (CURE), Maldonado, Uruguay.

## Contents:

- [Context of the talk](#Context-of-the-talk)
- [XFZ18 Algorithm](#XFZ18-Algorithm)
- [Implementation using SumOfSquares](#Implementation-using-SumOfSquares)
- [Results](#Results)

---

- JuMP/MOI have a great *expressive power*.
- `PolyJuMP` and `SumOfSquares` JuMP extensions are easy to use and efficient, useful for control theory and optimization modeling.
- Research problem: reachability analysis of dynamical systems. 
    - *Given a model of a system, exhaustively and automatically check whether this model meets a given specification.*

## Context of the talk

Textbook definition:

Consider a dynamical system $x'(t) = f(x(t), u(t), p(t))$, where:

- $t$ is time, say $t \in [0, T]$ for some $T \geq 0$ (possibly $+\infty$)
- $u$ is the input bounded by a set $u \in \mathcal{U} \subset \mathbb{R}^m$
- $p$ is a vector of possibly time-varying parameters, $p \in \mathcal{P} \in \mathbb{R}^p$

The reachable set at a certain point in time, $r$, is defined as the union of possible system states at $t = r$:

$$
\mathcal{R}(\delta) = \left\{ x(\delta) = \int_0^\delta f(x(t), u(t), p(t)) dt ~:~x(t) \in \mathcal{X}_0,~~u([0, \delta]) \in \mathcal{U},~~p([0, \delta]) \in \mathcal{P}\right\}.
$$
Here $u([0, \delta]) := \bigcup\limits_{t \in [0, \delta]} u(t)$.

The reachable set for a time interval is defined as the union of reachable sets at points in time withing the interval $t \in [0, \delta]$:

$$
\mathcal{R}([0, \delta]) = \bigcup\limits_{t \in [0, \delta]} \mathcal{R}(t).
$$

The purpose of reachability analysis is to reason about the set $\mathcal{R}([0, T])$ given $T \geq 0$. This set is often called the *flowpipe* of the dynamical system, which can only be computed approximately.

This notion extends naturally to *hybrid* dynamical systems, that is, described by continuous systems "connected" by discrete transitions.

---

Simplest case: take $x'(t) = Ax(t)$, whose solution for a given $x_0 \in \mathcal{X}_0$ is $x(t) = e^{At}x_0$.

![property](figures/NonValidatedFP.png)

![property](figures/ValidatedFP.png)

Research in last decade of so in this domain has led to:

- wrapping free algorithms 
- can handle discrete switchings
- methods that scale very well with the sytem's dimension

![reach_homog_boxes](figures/reach_homog_boxes.png)

![reach_homog_zonotopes](figures/reach_homog_zonotopes.png)

[LazySets.jl](https://github.com/JuliaReach/LazySets.jl/)

![lazysets](figures/LazySets.png)

![JuliaReach](figures/JuliaReachpaper_short.png)

![Reachability](figures/Reachability.png)

### Motivations

What are some **applications** of these methods?

- Cyber-physical systems
    - Systems that combine communication, control, and physical processes.
    - Can be safety-critical or mission-critical
    - Can have several interacting components.

Why?

- Formal verification can provide performance and correctness guarantees.

 
Examples:

- Mechanical Engineering (Building, Gearbox)
- Self-driving vehicles driving (Platoon)
- Aeronautics (Spacecraft rendez-vous)

- Related domain: Verification of deep neural networks, see [NeuralVerif.jl](https://github.com/sisl/NeuralVerification.jl)

![property](figures/Property.png)

Building benchmark:

- The building model is a model of the Los Angeles University Hospital with 8 floors, each of which has 3 degrees of freedom.
- It has 48 state variables in which we are mostly interested in the twenty-fifth state x25(t), which is the motion of the first coordinate.
- The twenty-fifth state is the interested output of the building model and should not reach a given unsafe region.
![Building](figures/Building.png)

Spacecraft rendez-vous benchmark:

- Spacecraft rendezvous is a perfect use case for formal verification of hybrid systems since mission failure can cost lives and is extremely expensive.
- The hybrid nature of this benchmark originates from a switched controller, while the dynamics of the spacecraft is purely continuous. In particular, the modes are approaching (100m-1000m), rendezvous attempt (less than 100m), and aborting.

![SpacecraftRendezVous](figures/SpacecraftRendezVous.png)

Platooning benchmark:

- The platooning benchmark considers a platoon of three vehicles following each other. This benchmark considers loss of communication between vehicles.

![Platooning](figures/Platooning.png)

![juliareachorg](figures/JuliaReachOrg.png)

See the github org [JuliaReach.org](juliareach.org) or contact us in the [gitter channel](https://gitter.im/JuliaReach/Lobby) if you want to know more.

## XFZ18 Algorithm

So far so good.. How about systems of **polynomial** ODEs?

$$
x'(t) = f(x(t)),   \textrm{where f is a polynomial map}
$$

Several methods exist, such as simulation-based methods: Zonotope-based, Taylor models, HJE, simulation-based, hybridization, ... (see, e.g. [ARCH report in the NLN category](https://easychair.org/publications/paper/gjfh)).

In the rest of this notebook we consider the method from [1]:

- Produces a sequence of inner-approximations with guarantee of convergence to the exact reachable set,
- Consider the *Hamilton-Jacobi-Bellman equation*:

$$
\mathcal{L}\Phi : \dfrac{\partial \Phi(x, t)}{\partial t} + \dfrac{\partial \Phi(x, t)}{\partial x} f(x) = 0,
$$
with initial value $\Phi(x, 0) = V_0(x)$.

- It is shown in [1] that it can be solved by a converging hierarchy of semidefinite programs.
-  We obtain a monotonically decreasing sequence of infima $\epsilon$ that characterize the discrepancy between the approximate solution to the HJE and its exact solution over $\mathcal{X} \times [0, T]$ with the relaxation order increasing

---

The methods allows to obtain an under (also an over) approximation of the reach set at a given time by solving a semi-definite program.

Assumptions:

- the right-hand side $f(x)$ is polynomial
- initial set is given as a basic semialgebraic set $\mathcal{X}_0 = \{x  : V_0(x) \le 0\}$ compact set, $V_0(x)$ multivariate polynomial
- we have an a-priori enclosing set $\mathcal{Y} = \{x : g(x) \ge 0\}$


[1] Xue, B., Fränzle, M., & Zhan, N. (2018, April). [Under-Approximating Reach Sets for Polynomial Continuous Systems. In Proceedings of the 21st International Conference on Hybrid Systems: Computation and Control (part of CPS Week) (pp. 51-60). ACM.](https://dl.acm.org/citation.cfm?id=3178133)

[2] Korda, Milan, Didier Henrion, and Colin N. Jones. "Inner approximations of the region of attraction for polynomial dynamical systems." [IFAC Proceedings Volumes 46.23 (2013): 534-539.](https://www.sciencedirect.com/science/article/pii/S1474667016317153).

Optimization problem:

![Program](figures/Program.png)

**Theorem:** The set $\{x : \Phi(x, \tau) \le 0\}$ is an under-approximation of the reachable set for $\tau \in [t_0, T]$.

---

#### SOS relaxation

![Program](figures/SOS.png)

## Modeling with JuMP

Consider the following [Van der Pol oscillator](https://en.wikipedia.org/wiki/Van_der_Pol_oscillator):

$$
\begin{array}
\dot{x}_1 &= x_2 \\
\dot{x}_2 &= -0.2x_1 + x_2 - 0.2x_1^2 x_2
\end{array}
$$
The Van der Pol oscillator was introduced by the Dutch physicist Balthasar van der Pol. It's often considered as the first benchmark for a reachability or invariant computation method on nonlinear dynamics.

It's interesting because:

- The system has a stable limit cycle however shows complicated behavior.
- If you could not find a fixed point in the first period then you probably may never find one.
- The flowpipe construction methods suffer from may suffer from wrapping effect.

However:

- Can use subdivision in 2D to obtain a better accuracy, it is not expensive.
- May use different heuristics to improve performance and accuracy that only work in 2D.

In [1]:
# load packages
using MultivariatePolynomials,
      JuMP,
      PolyJuMP,
      SumOfSquares,
      DynamicPolynomials,
      MathematicalSystems,
      MosekTools
 
const ∂ = differentiate

differentiate (generic function with 18 methods)

In [25]:
# symbolic variables
@polyvar x₁ x₂ t

(x₁, x₂, t)

(Tip: type subindices with `x\_1[TABl]`)

In [26]:
typeof(x₁)

PolyVar{true}

In [27]:
supertype(typeof(x₁))

MultivariatePolynomials.AbstractVariable

In [28]:
# time duration (scaled, see dynamics below)
T = 1.0 

# dynamics
f = 2 * [x₂, -0.2*x₁ + x₂ - 0.2*x₁^2*x₂] 

2-element Array{Polynomial{true,Float64},1}:
 2.0x₂                    
 -0.4x₁²x₂ - 0.4x₁ + 2.0x₂

In [29]:
# set of initial states X₀ = {x: V₀(x) <= 0}
V₀ = x₁^2 + x₂^2 - 0.25

x₁² + x₂² - 0.25

In [30]:
typeof(V₀)

Polynomial{true,Float64}

In [31]:
# constraints Y = {x: g(x) >= 0} compact search space Y x [0, T]
g = 25 - x₁^2 - x₂^2

-x₁² - x₂² + 25

In [85]:
# degree of the relaxation
k = 12

12

In [86]:
# monomial vector up to order k, 0 <= sum_i alpha_i <= k, if alpha_i is the exponent of x_i
X = monomials([x₁, x₂], 0:k)
XT = monomials([x₁, x₂, t], 0:k)

455-element MonomialVector{true}:
 x₁¹²    
 x₁¹¹x₂  
 x₁¹¹t   
 x₁¹⁰x₂² 
 x₁¹⁰x₂t 
 x₁¹⁰t²  
 x₁⁹x₂³  
 x₁⁹x₂²t 
 x₁⁹x₂t² 
 x₁⁹t³   
 x₁⁸x₂⁴  
 x₁⁸x₂³t 
 x₁⁸x₂²t²
 ⋮       
 x₂t²    
 t³      
 x₁²     
 x₁x₂    
 x₁t     
 x₂²     
 x₂t     
 t²      
 x₁      
 x₂      
 t       
 1       

In [87]:
# create a SOS JuMP model to solve with Mosek
model = SOSModel(with_optimizer(Mosek.Optimizer))

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Mosek

In [88]:
# add unknown Φ to the model
@variable(model, Φ, Poly(XT));

In [89]:
# jacobian
∂t = α -> ∂(α, t)
∂xf = α -> ∂(α, x₁) * f[1] + ∂(α, x₂) * f[2] 
LΦ = ∂t(Φ) + ∂xf(Φ);

In [90]:
# Φ(x, t) at time 0
Φ₀ = subs(Φ, t => 0.);

In [91]:
# scalar variable
@variable(model, ϵ)

ϵ

In [92]:
dom1 = @set t*(T-t) >= 0 && g >= 0

SemialgebraicSets.BasicSemialgebraicSet{Float64,Polynomial{true,Float64},SemialgebraicSets.AlgebraicSet{Float64,Polynomial{true,Float64},SemialgebraicSets.Buchberger,SemialgebraicSets.SolverUsingMultiplicationMatrices{SemialgebraicSets.GröbnerBasisMultiplicationMatricesAlgorithm,SemialgebraicSets.ReorderedSchurMultiplicationMatricesSolver{Float64,Random.MersenneTwister}}}}(Algebraic Set defined by no equality
, Polynomial{true,Float64}[-t² + t, -x₁² - x₂² + 25.0])

In [93]:
dom1.p

2-element Array{Polynomial{true,Float64},1}:
 -t² + t          
 -x₁² - x₂² + 25.0

In [94]:
dom2 = @set g >= 0

SemialgebraicSets.BasicSemialgebraicSet{Float64,Polynomial{true,Float64},SemialgebraicSets.AlgebraicSet{Float64,Polynomial{true,Float64},SemialgebraicSets.Buchberger,SemialgebraicSets.SolverUsingMultiplicationMatrices{SemialgebraicSets.GröbnerBasisMultiplicationMatricesAlgorithm,SemialgebraicSets.ReorderedSchurMultiplicationMatricesSolver{Int64,Random.MersenneTwister}}}}(Algebraic Set defined by no equality
, Polynomial{true,Float64}[-x₁² - x₂² + 25.0])

In [95]:
@constraint(model, ϵ >= 0.)

ϵ ≥ 0.0

In [96]:
# defining the constraints
@constraint(model, ϵ >= 0.)
@constraint(model, LΦ ∈ SOSCone(), domain = dom1)
@constraint(model, ϵ - LΦ ∈ SOSCone(), domain = dom1)
@constraint(model, Φ₀ - V₀ ∈ SOSCone(), domain = dom2)
@constraint(model, ϵ + V₀ - Φ₀ ∈ SOSCone(), domain = dom2)

@objective(model, Min, ϵ)

ϵ

In [97]:
optimize!(model)

println("Relaxation order : k = $k")
println("JuMP.termination_status(model) = ", JuMP.termination_status(model))
println("JuMP.primal_status(model) = ", JuMP.primal_status(model))
println("JuMP.dual_status(model) = ", JuMP.dual_status(model))
println("JuMP.objective_bound(model) = ", JuMP.objective_bound(model))
println("JuMP.objective_value(model) = ", JuMP.objective_value(model))

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 1544            
  Cones                  : 0               
  Scalar variables       : 456             
  Matrix variables       : 10              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimizat

In [98]:
MOI.get(model, MOI.SolveTime())

STALL


5.063136100769043

## Results

### Comparison with an implementation in MATLAB/YALMIP

[YALMIP](https://yalmip.github.io/) is a MATLAB based modelling language, suitable for a large class of problems in control theory and optimization. The 2004 IEEE ICRA tool paper [YALMIP: A toolbox for modeling and optimization in MATLAB](https://ieeexplore.ieee.org/document/1393890) counts with > 6k citations in google scholar.

#### Matlab code:

```matlab
sdpvar x y t obj;
f=2*[y;(-0.2*x+y-0.2*x^2*y);0.5];
T=1;
V0=x^2+y^2-0.25;
g=25-x^2-y^2;
degree = 12;

x1=[x y t];
[Phi,coe]= polynomial(x1,degree);
Phi_0=replace(Phi,[t],[0]);
Phi_derivative=jacobian(Phi,x1)*f;
degree1=degree;
 
[s1,coe1]=polynomial(x1,degree1);
[s2,coe2]=polynomial(x1,degree1);
[s4,coe4]=polynomial(x1,degree1);
[s5,coe5]=polynomial(x1,degree1);
[s7,coe7]=polynomial([x y],degree1);
[s9,coe9]=polynomial([x y],degree1);

F=[sos(Phi_derivative-s1*t*(T-t)-s2*g),sos(obj-Phi_derivative-s4*t*(T-t)-s5*g),sos(Phi_0-V0-s7*g),...
   sos(obj+V0-Phi_0-s9*g),sos(s1),sos(s2),sos(s4),sos(s5),sos(s7),sos(s9),obj>=0];
 
 ops = sdpsettings('solver','mosek','sos.newton',1,'sos.congruence',1);
 diagnostics= solvesos(F,obj,ops,[obj;coe;coe1;coe2;coe4;coe5;coe7;coe9])
```

23 LOC

#### Julia code:

```julia
@polyvar x₁ x₂ t
f = 2 * [x₂, -0.2*x₁ + x₂ - 0.2*x₁^2*x₂] 
T = 1.0 
V₀ = x₁^2 + x₂^2 - 0.25
g = 25 - x₁^2 - x₂^2
k = 12

X = monomials([x₁, x₂], 0:k)
XT = monomials([x₁, x₂, t], 0:k)

model = SOSModel(with_optimizer(Mosek.Optimizer))
@variable(model, Φ, Poly(XT))

∂xf = α -> ∂(α, x₁) * f[1] + ∂(α, x₂) * f[2] 
LΦ = ∂(Φ, t) + ∂xf(Φ)

Φ₀ = subs(Φ, t => 0.)
@variable(model, ϵ)
dom1 = @set t*(T-t) >= 0 && g >= 0
dom2 = @set g >= 0
@constraint(model, ϵ >= 0.)
@constraint(model, LΦ ∈ SOSCone(), domain = dom1)
@constraint(model, ϵ - LΦ ∈ SOSCone(), domain = dom1)
@constraint(model, Φ₀ - V₀ ∈ SOSCone(), domain = dom2)
@constraint(model, ϵ + V₀ - Φ₀ ∈ SOSCone(), domain = dom2)

@objective(model, Min, ϵ)
optimize!(model)
```

~~24 LOC~~ => 23 LOC

### Comparison of the generated model and runtime

|  Package    | k    |Constraints|Scalar variables|Matrix variables|Time(s)|
|-------------|------|-----------|----------------|----------------|-------|
|**SumOfSquares v0.3.0**         |    2 |    83    |      13       |            8   |   < 1 |
|YALMIP       |    2 |       152 |             63 |              10|   < 1 |
||
|**SumOfSquares v0.3.0**         |    3 |    199    |        21     |        10       |   < 1 |
|YALMIP       |    3 |       254 |            121 |              10|   ~ 1 |
||
|**SumOfSquares v0.3.0**         |    4 |   199     |       36      |      10         |   < 1 |
|YALMIP       |    4 |       394 |           206  |              10|  1.18 |
||
|**SumOfSquares v0.3.0**         |    5 |     387   |       57      |         10      |   < 1 |
|YALMIP       |    5 |      578  |         323     |          10    |  0.11 |
||
|**SumOfSquares v0.3.0**         |    6 |    387    |       85      |       10        |   < 1 |
|YALMIP       |    6 |    812    |         477    |    10          |  1.10  |
||
|**SumOfSquares v0.3.0**         |    7 |   663     |     121        |      10         |   < 1 |
|YALMIP       |    7 |     1102   |        673     |        10      |  1.52 |
||
|**SumOfSquares v0.3.0**         |    8 |  663      |        166     |           10    |   < 1 |
|YALMIP       |    8 |    1454    |       916       |          10    |  1.10 |
||
|**SumOfSquares v0.3.0**         |    9 |      1043  |        221     |        10       |   1.70 |
|YALMIP       |    9 |   1874     |       1211     |         10     |  1.58  |
||
|**SumOfSquares v0.3.0**         |    10 |    1043    |    287         |     10          |   1.67 |
|YALMIP       |   10 |     2368   |       1563     |        10      |  2.73 |
||
|**SumOfSquares v0.3.0**         |    11 |   1543     |      365       |     10          |   4.88 |
|YALMIP       |   11 |   2942    |       1977      |       10       |  2.30 |
||
|**SumOfSquares v0.3.0**         |    12 | 1543       |   456          |         10      |  5.02 |
|YALMIP       |   12 |    3602    |      2458       |      10        | 6.57  |


Using: `MosekTools v0.7.0`, `SumOfSquares v0.3.0`.

Why are the generated models different?

- Recent fix gave a massive improvement: https://github.com/JuliaOpt/MosekTools.jl/issues/11
- With SumOfSquares one can directly create sum of squares variables. With YALMIP one first creates a polynomial, and the constrain the scalar variables to be SOS. 

Comparison:

- smaller problem instances may have an impact in *performance* (see below!)
- free for people doing research (i have a Mosek academic license)

### Extracting the solution and visualization 

- The sublevel sets $Φ(x, T) \le 0$ provide an *under-approximation* of the reachset at time `t=T`,
- while $Φ(x, T) - ϵopt * (T+1) \le 0$ gives an *over-approximation.*

In [99]:
JuMP.value(model[:Φ])

STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STAL

-3.254406926195832e-6x₁¹² + 3.834217164793429e-6x₁¹¹x₂ + 1.656962882210182e-20x₁¹¹t - 6.3384127954307375e-6x₁¹⁰x₂² + 1.3171927844747763e-20x₁¹⁰x₂t + 0.00017977769422352572x₁¹⁰t² + 1.7821158886139766e-6x₁⁹x₂³ - 4.9752291830315605e-20x₁⁹x₂²t - 0.0003404492894529957x₁⁹x₂t² + 3.836278593253594e-19x₁⁹t³ - 3.874093809241813e-7x₁⁸x₂⁴ + 3.938591835332774e-20x₁⁸x₂³t + 0.00017172005010559737x₁⁸x₂²t² - 4.199894157533322e-19x₁⁸x₂t³ + 0.00022253181841657853x₁⁸t⁴ + 1.5483592082483843e-6x₁⁷x₂⁵ - 5.2616440647594625e-20x₁⁷x₂⁴t + 0.000259547136068073x₁⁷x₂³t² - 2.6658461147986677e-19x₁⁷x₂²t³ + 0.000406478741702339x₁⁷x₂t⁴ - 5.067988419296385e-18x₁⁷t⁵ - 1.7690055890869638e-6x₁⁶x₂⁶ + 2.7396567382407404e-20x₁⁶x₂⁵t - 0.0004292363708925713x₁⁶x₂⁴t² + 1.5685783222264855e-18x₁⁶x₂³t³ - 0.0011434231217147754x₁⁶x₂²t⁴ + 2.769597746752094e-18x₁⁶x₂t⁵ - 0.008524441229491156x₁⁶t⁶ - 9.798790301107213e-7x₁⁵x₂⁷ + 6.28066342893135e-20x₁⁵x₂⁶t + 4.1098887836025915e-5x₁⁵x₂⁵t² + 6.930368275509355e-19x₁⁵x₂⁴t³ + 0.0004939710550388

In [100]:
# Recovering the solution:
ϵopt = JuMP.objective_value(model)

STALL


0.02305835379792861

In [123]:
# Punder <= 0
Punder = subs(JuMP.value(model[:Φ]), t => T);

STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STAL

In [122]:
Punder.x

91-element MonomialVector{true}:
 x₁¹²   
 x₁¹¹x₂ 
 x₁¹⁰x₂²
 x₁⁹x₂³ 
 x₁⁸x₂⁴ 
 x₁⁷x₂⁵ 
 x₁⁶x₂⁶ 
 x₁⁵x₂⁷ 
 x₁⁴x₂⁸ 
 x₁³x₂⁹ 
 x₁²x₂¹⁰
 x₁x₂¹¹ 
 x₂¹²   
 ⋮      
 x₁x₂³  
 x₂⁴    
 x₁³    
 x₁²x₂  
 x₁x₂²  
 x₂³    
 x₁²    
 x₁x₂   
 x₂²    
 x₁     
 x₂     
 1      

In [102]:
# Pover <= 0
Pover = subs(JuMP.value(model[:Φ]), t => T) - ϵopt * (T+1);

STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STAL

### Using `ImplicitEquations`

- [ImplicitEquations.jl](https://github.com/jverzani/ImplicitEquations.jl)
    - Method to graph two-dimensional implicit equations and inequalities.
    - Native Julia implementation of Tupper's [Reliable Two-Dimensional Graphing Methodsfor Mathematical Formulae with Two Free Variables](http://www.dgp.toronto.edu/people/mooncake/papers/SIGGRAPH2001_Tupper.pdf).
    

- We use static polynomials from [StaticPolynomials](https://github.com/JuliaAlgebra/StaticPolynomials.jl) for fast   evaluation:
    - Library for fast evaluation of multivariate polynomials.
    - `StaticPolynomials` encodes information on the *type signature* which terms are non-zero.
    - Uses Julia's metaprogramming capabilities and generated functions.

In [116]:
using StaticPolynomials

# convert to a static polynomial for faster evaluation
Punder_st = StaticPolynomials.Polynomial(Punder)
Pover_st = StaticPolynomials.Polynomial(Pover)

_Punder(x, y) = Punder_st([x, y])
_Pover(x, y) = Pover_st([x, y])

_Pover (generic function with 1 method)

In [117]:
@btime Punder(1.0, 1.0)

  2.125 μs (10 allocations: 432 bytes)


-0.18743496973607388

In [118]:
@btime Punder_st([1.0, 1.0])

  97.046 ns (2 allocations: 112 bytes)


-0.1874349697360741

In [119]:
2.125e-6 / 97.05e-9

21.895929933024213

In [120]:
using ImplicitEquations, Plots
gr()

Plots.GRBackend()

In [ ]:
G = plot()
plot!(G, _Punder ⩵ 0., xlims=(-4, 4), ylims=(-4, 4), color="red")
plot!(G, _Pover ⩵ 0., xlims=(-4, 4), ylims=(-4, 4), color="blue")
G

![k12_g](figures/k12_g.png)

### Using `IntervalConstraintProgramming` to obtain a rigorous enclosure of the feasible set

- [IntervalConstraintProgramming.jl](https://github.com/JuliaIntervals/IntervalConstraintProgramming.jl) is a package that allows to calculate rigorously the feasible region for a set of inequalities with Julia:

    - Constraints on real-valued variables given by an arbitrary Julia function.
    - Multi-dimensional interval boxes implemented in `IntervalArithmetic.jl`.
    - Calculates inner and outer approximations to feasible set (i.e. the set that satisfies the constraints).

In [80]:
include("constraints.jl")

In [103]:
S = Separator(-Inf..0.0, (x, y) -> Punder(x, y))

Separator:
  - variables: x, y
  - expression: (((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((-3.254406926195832e-6 * x ^ 12 + 3.834217164793429e-6 * (x ^ 11 * y ^ 1)) + -6.3384127954307375e-6 * (x ^ 10 * y ^ 2)) + 1.7821158886139766e-6 * (x ^ 9 * y ^ 3)) + -3.874093809241813e-7 * (x ^ 8 * y ^ 4)) + 1.5483592082483843e-6 * (x ^ 7 * y ^ 5)) + -1.7690055890869638e-6 * (x ^ 6 * y ^ 6)) + -9.798790301107213e-7 * (x ^ 5 * y ^ 7)) + 5.210797975903119e-7 * (x ^ 4 * y ^ 8)) + 1.3500407479164792e-6 * (x ^ 3 * y ^ 9)) + 4.4924259835334384e-7 * (x ^ 2 * y ^ 10)) + -7.166251275333759e-8 * (x ^ 1 * y ^ 11)) + -1.5032013847766846e-7 * (x ^ 0 * y ^ 12)) + 1.4831621204102143e-20 * x ^ 11) + 1.3691290786000017e-20 * (x ^ 10 * y ^ 1)) + -5.0093500950317737e-20 * (x ^ 9 * y ^ 2)) + 4.3677787535910767e-20 * (x ^ 8 * y ^ 3)) + -3.173767662076143e-20 * (x ^ 7 * y ^ 4)) + 1.9724120886883362e-20 * (x ^ 6 * y ^ 5)) + 3.932019772930437e-20 * (x ^ 5 * y ^ 6)) + -3.366118

In [108]:
p = pave(S, IntervalBox(-2..2, 2), 0.01)

Paving:
- tolerance ϵ = 0.01
- inner approx. of length 14983
- boundary approx. of length 27042

In [110]:
using Plots

plot(p.inner, lw=0, aspectratio=1, label="inner")
plot!(p.boundary, lw=0, label="boundary", legend=:topleft)

![k12_a](figures/k12_a.png)

![k12_b](figures/k12_b.png)

![k12_c](figures/k12_c.png)

![k12_d](figures/k12_d.png)

In [ ]:
ϵ = 0.01
S = Separator(-Inf..0.0, Punder_mt)
p = pave(S, IntervalBox(-2..2, 2), ϵ)

Pover_mt = tomtexpr(Pover)
Sover = Separator(-Inf..0.0, Pover_mt)
pover = pave(Sover, IntervalBox(-2..2, 2), ϵ)

plot(p.inner, lw=0, aspectratio=1, label="under", alpha=.4)
plot!(pover.inner, lw=0, label="over", legend=:topleft, alpha=.4)

![k12_f](figures/k12_f.png)

### Laub-Loomis

Consider the Laub-loomis benchmark the from ARCH NLN problem set.

In [12]:
@polyvar x₁ x₂ x₃ x₄ x₅ x₆ x₇ t
x = [x₁, x₂, x₃, x₄, x₅, x₆, x₇]

f = [1.4x₃ - 0.9x₁,
     2.5x₅ - 1.5x₂,
     0.6x₇ - 0.8x₂*x₃,
     2 - 1.3x₃*x₄,
     0.7x₁ - x₄*x₅,
     0.3x₁ - 3.1x₆,
     1.8x₆ - 1.6x₂*x₇]

7-element Array{Polynomial{true,Float64},1}:
 -0.9x₁ + 1.4x₃  
 -1.5x₂ + 2.5x₅  
 -0.8x₂x₃ + 0.6x₇
 -1.3x₃x₄        
 -x₄x₅ + 0.7x₁   
 0.3x₁ - 3.1x₆   
 -1.6x₂x₇ + 1.8x₆

In [17]:
T = 1.0 
V₀ = x₁^2 + x₂^2 + x₃^2 + x₄^2 + x₅^2 + x₆^2 + x₇^2 - 0.1
g = 25 - (x₁^2 + x₂^2 + x₃^2 + x₄^2 + x₅^2 + x₆^2 + x₇^2)
k = 12

X = monomials([x₁, x₂, x₃, x₄, x₅, x₆, x₇], 0:k)
XT = monomials([x₁, x₂, x₃, x₄, x₅, x₆, x₇, t], 0:k)

125970-element MonomialVector{true}:
 x₁¹²    
 x₁¹¹x₂  
 x₁¹¹x₃  
 x₁¹¹x₄  
 x₁¹¹x₅  
 x₁¹¹x₆  
 x₁¹¹x₇  
 x₁¹¹t   
 x₁¹⁰x₂² 
 x₁¹⁰x₂x₃
 x₁¹⁰x₂x₄
 x₁¹⁰x₂x₅
 x₁¹⁰x₂x₆
 ⋮       
 x₇²     
 x₇t     
 t²      
 x₁      
 x₂      
 x₃      
 x₄      
 x₅      
 x₆      
 x₇      
 t       
 1       

In [18]:
model = SOSModel(with_optimizer(Mosek.Optimizer))

@variable(model, Φ, Poly(XT));

∂xf = α -> sum([∂(α, x[i]) * f[i] for i in 1:7])
LΦ = ∂(Φ, t) + ∂xf(Φ);

model = SOSModel(with_optimizer(Mosek.Optimizer))

@variable(model, Φ, Poly(XT))
Φ₀ = subs(Φ, t => 0.)
@variable(model, ϵ)
dom1 = @set t*(T-t) >= 0 && g >= 0
dom2 = @set g >= 0
@constraint(model, ϵ >= 0.)
@constraint(model, LΦ ∈ SOSCone(), domain = dom1)
@constraint(model, ϵ - LΦ ∈ SOSCone(), domain = dom1)
@constraint(model, Φ₀ - V₀ ∈ SOSCone(), domain = dom2)
@constraint(model, ϵ + V₀ - Φ₀ ∈ SOSCone(), domain = dom2)

@objective(model, Min, ϵ)
optimize!(model)

VariableNotOwned{VariableRef}: VariableNotOwned{VariableRef}(noname)

### Lotka-Volterra

- 3D model

In [20]:
@time begin
    # symbolic variables
    @polyvar x₁ x₂ x₃ t

    # time duration (scaled, see dynamics below)
    T = 1.0 

    # dynamics
    f = 3 * [-x₁*x₂ + x₁*x₃, -x₂*x₃ + x₂*x₁, -x₃*x₁ + x₃*x₂]

    # set of initial states X₀ = {x: V₀(x) <= 0}
    V₀ = 100*(x₁^2 + x₂^2 + x₃^2) - 1.0

    # constraints Y = {x: g(x) >= 0} compact search space Y x [0, T]
    g = 0.16 - (x₁^2 + x₂^2 + x₃^2)

    # degree of the relaxation
    k = 12

    # monomial vector up to order k
    X = monomials([x₁, x₂, x₃], 0:k)
    XT = monomials([x₁, x₂, x₃, t], 0:k)

    # create a SOS JuMP model to solve with Mosek
    model = SOSModel(with_optimizer(Mosek.Optimizer))

    # add unknown Φ to the model
    @variable(model, Φ, Poly(XT))

    # jacobian
    ∂xf = α -> sum([∂(α, x[i]) * f[i] for i in 1:3])
    LΦ = ∂t(Φ) + ∂xf(Φ)

    # Φ(x, t) at time 0
    Φ₀ = subs(Φ, t => 0.)

    # scalar variable
    @variable(model, ϵ)

    dom1 = @set t*(T-t) >= 0 && g >= 0
    dom2 = @set g >= 0
    @constraint(model, ϵ >= 0.)
    @constraint(model, LΦ ∈ SOSCone(), domain = dom1)
    @constraint(model, ϵ - LΦ ∈ SOSCone(), domain = dom1)
    @constraint(model, Φ₀ - V₀ ∈ SOSCone(), domain = dom2)
    @constraint(model, ϵ + V₀ - Φ₀ ∈ SOSCone(), domain = dom2)

    @objective(model, Min, ϵ)

    optimize!(model)

    MOI.get(model, MOI.SolveTime())
end

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 4551            
  Cones                  : 0               
  Scalar variables       : 1821            
  Matrix variables       : 10              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimizat

19.217565059661865

In [21]:
ϵopt = JuMP.objective_value(model)

1.9392669478552092e-6

Comparison:

```julia
YALMIP SOS module started...
-------------------------------------------------------------------------
Detected 11375 parametric variables and 4 independent variables.
```

![table](figures/table.png)

---

### End notes

- `SumOfSquares.jl` extension of JuMP has a great expressive power.
- The syntax for defining set-based constraints is intuitive and easy to use.
- There is low overhead for model generation, and the size of generated models is competitive with established tools.

#### Future work

- Tooling ([Reachability.jl](https://github.com/JuliaReach/Reachability.jl)).
- Implement methods for property checking.
- Extend to hybrid systems.

![juliareachorg](figures/JuliaReachOrg.png)